In [1]:
pip install dropbox

Note: you may need to restart the kernel to use updated packages.


In [2]:
import dropbox
import pandas as pd
import io

import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Time Series analysis
# https://www.machinelearningplus.com/time-series/time-series-analysis-python/
from dateutil.parser import parse
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import numpy as np
import pandas as pd
plt.rcParams.update({'figure.figsize': (10, 7), 'figure.dpi': 120})



In [124]:
# Important:
# 1. Save a copy of this script to your own Google Drive
# 2. Genereate your own DROPBOX_TOKEN,
#    NEVER share it with others or check into version control systems like GitHub
dbx = dropbox.Dropbox('nsLc8YQnhKQAAAAAAAAAAUhNcy49b5vPam_1YlmP_16UdsYds9pORaTbOpGQ-XMy') ## REPLACE TOKEN HERE
dbx.users_get_current_account()
data_blank = [[]]
df_main = pd.DataFrame(data_blank)
df_logs = pd.DataFrame(data_blank)
df_grades = pd.DataFrame(data_blank)

# define function to parse term code from file path string
def set_term_code(file_path,year) :
    term_code = ''
    if 'fall' in file_path :
        term_code = year + '10'
        term_code = int(term_code) + 100
        term_code = str(term_code)
    elif 'winter' in file_path :
        term_code = year + '20'
    else :
        print('Error reading term description from file path')
    return term_code

# def get_term_desc(term_code) :
#     int(term_code)

for entry in dbx.files_list_folder('/MyLA Data Identified Subset').entries:
    file_path = entry.path_lower
    
    
    
    # check for course id map file name to save dataframe for later joining
    if 'course_map' in file_path and 'winter2019' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
#             df = pd.read_csv(stream, header=None)
            df = pd.read_csv(stream)
        df_cmap = df
        print(' - - - course map file')
        df_cmap = df_cmap.rename({'Canvas Course ID': 'Canvas CID','Canvas Course Name':'Canvas CName'}
                                 , axis='columns')
        df_cmap['Canvas CID'] = df_cmap['Canvas CID'].astype(str)
        
        
        
    # check for student id map file name to save dataframe for later joining
    if 'student_map' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
#             df = pd.read_csv(stream, header=None)
            df = pd.read_csv(stream)
        df_smap = df
        print(' - - - student map file')
#         print(file_path)
        df_smap.columns = ['User ID','Uniqname']
        
        
        

    # check for student grades data and compile into one frame
    elif 'canvasgrades' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        pd.set_option('display.max_columns', None)
        year_code = file_path[int(file_path.find('.csv'))-4:int(file_path.find('.csv'))]
        print(' - - - grades file')
#         print(file_path)
        df_term = set_term_code(file_path,year_code)
#         print('Term is: ', df_term)
        df['Term Code'] = df_term
        
        df = df.rename({'Canvas_course_id': 'Canvas CID','final_score':'Final Grade',
                        'unique_name':'Uniqname'}, axis='columns')
        df['Canvas CID'] = df['Canvas CID']%1000000
        df['Canvas CID'] = df['Canvas CID'].astype(str)
        df_grades = pd.concat([df_grades,df])
        
        
        

    # check for student course roster data file name to compile course headcount main dataframe
    elif 'myla_student_data' in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        pd.set_option('display.max_columns', None)
        year_code = file_path[int(file_path.find('_withcanvas'))-4:int(file_path.find('_withcanvas'))]
        print(' - - - student course file')
#         print(file_path)
        df_term = set_term_code(file_path,year_code)
#         print('Term is: ', df_term)
        df['Term Code'] = df_term
        if df_term == '201920' :
            df = df.rename({'Canvas Course': 'CanvasCourseName'}, axis='columns')
            df['CanvasCourseID_short'] = ' '
        if df_term == '202020' or df_term == '202110' :
            df = df.rename({'CanvasCousreName': 'CanvasCourseName'}, axis='columns')
        df = df[['Campus ID', 'Sex', 'International or Domestic','Acad Level BOT', 'Cum GPA', 
                   'CanvasCourseName', 'CanvasCourseID_short', 'Term Code']]
        df = df.rename({'CanvasCourseName': 'Canvas CName','International or Domestic':'Intl/Dom',
                           'Acad Level BOT':'Class Level','CanvasCourseID_short':'Canvas CID',
                           'Campus ID':'Uniqname'}, axis='columns')
        df['Canvas CID'] = df['Canvas CID'].astype(str)
        df['Uniqname'] = df['Uniqname'].str.lower()
        df_main = pd.concat([df_main,df])
#         print(df.head(5))
#         print(df.columns)
    
        
    # check for myla log data file name to analyze tool usage data, later linked to student course data
    elif 'myla_event_log' in file_path and 'grade' not in file_path :
        _, res = dbx.files_download(file_path)
        with io.BytesIO(res.content) as stream:
            df = pd.read_csv(stream)
        pd.set_option('display.max_columns', None)
        year_code = file_path[int(file_path.find('.csv'))-4:int(file_path.find('.csv'))]
        print(' - - - event log file')
        df_term = set_term_code(file_path,year_code)
#         print('Term is: ', df_term)
        df['Term Code'] = df_term
        df = df.rename({'id':'Click ID','timestamp':'Click Timestamp','action':'Module',
                       'username':'Uniqname','course_id':'Canvas CID'}, axis='columns')
        df['Canvas CID'] = df['Canvas CID']%1000000
        df['Canvas CID'] = df['Canvas CID'].astype(str)
        df['Click ID'] = df['Click ID'].astype(str)
        df = df[['Click ID', 'Click Timestamp', 'Module', 'Uniqname', 'Canvas CID', 'Term Code']]
        df_logs = pd.concat([df_logs,df])
#         print(df.head(5))
#         print(df_logs.columns)
#         print(df_logs.head(5))




    else:
        print(' - - - other file')
        print(file_path)
print('\nBREAK')





 - - - other file
/myla data identified subset/reference only
 - - - grades file
 - - - student course file
 - - - event log file
 - - - grades file
 - - - grades file
 - - - grades file
 - - - event log file
 - - - student course file
 - - - event log file
 - - - event log file
 - - - student course file
 - - - student course file
 - - - course map file
 - - - other file
/myla data identified subset/winter2019_course_map.csv
 - - - student map file

BREAK


In [125]:

df_main = pd.merge(df_main,df_cmap,on='Canvas CName',how='left')

In [126]:



# print(df_main['Canvas CID_x'])
# df_main['Canvas CID_x'] + df_main['Canvas CID_y']




df_main['Canvas CID'] = np.where(
    df_main['Canvas CID_y'].isna(), df_main['Canvas CID_x'], 
     np.where(df_main['Canvas CID_y'].notna(),  df_main['Canvas CID_y'], 'Unknown')
)
df_main.drop(columns=['Canvas CID_y','Canvas CID_x'],inplace=True)
df_main

# df_main = df_main.join(df_cmap,on='Canvas CName', how='left', lsuffix='', rsuffix=' map')
# df_main.to_clipboard()
# print(df_main)


,Uniqname,Sex,Intl/Dom,Class Level,Cum GPA,Canvas CName,Term Code,Canvas CID
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,abdsul,Male,International,Grad Mastr,3.673,HMP 654 001 FA 2019,202010,317173
2,aileenz,Female,Domestic,Grad Mastr,3.731,HMP 654 001 FA 2019,202010,317173
3,alexfox,Male,Domestic,Grad Mastr,3.877,HMP 654 001 FA 2019,202010,317173
4,angiemm,Female,Domestic,Grad Mastr,3.879,HMP 654 001 FA 2019,202010,317173
...,...,...,...,...,...,...,...,...
4719,vessecce,Male,Domestic,Sophomore,2.950,EECS 215 002 WN 2019,201920,268094
4720,xuanting,Male,Domestic,Junior,3.606,EECS 215 002 WN 2019,201920,268094
4721,yunzen,Male,International,Sophomore,3.430,EECS 215 002 WN 2019,201920,268094
4722,ziyangji,Male,International,Junior,3.837,EECS 215 002 WN 2019,201920,268094


In [130]:
# 324902
# 341239
df_grades['Uniqname'] = df_grades['Uniqname'].apply(lambda x:x.split("@")[0])

grades_names = df_grades['Uniqname'].unique()
grades_names = set(grades_names)
# print(grades_names)
main_names = df_main['Uniqname'].unique()
main_names = set(main_names)
# print(main_names)
combo_names = main_names.union(grades_names)
combo_names = pd.DataFrame(combo_names)
combo_names.to_clipboard()
# print(len(main_names.difference(grades_names)))
# print(len(grades_names.difference(main_names)))

# main
# grades_names
# combo_names = pd.merge(main_names,grades_names,how='left',indicator=True)
# combo_names.to_clipboard()



In [131]:
df_grades = pd.merge(df_grades,df_smap,on='Uniqname',how='left')

ValueError: You are trying to merge on object and int64 columns. If you wish to proceed you should use pd.concat

In [128]:

def get_term_desc(term_code) :
    term_type = int(term_code)%100
    year = math.trunc(int(term_code)/100)
    if term_type == 10 :
        year -= 1
        term_typ_desc = 'Fall'
    else :
        term_typ_desc = 'Winter'
    term_desc = term_typ_desc + ' ' + str(year)
    return(term_desc)



df_main = df_main.dropna()
df_main = df_main.drop_duplicates()
df_main = df_main[df_main['Canvas CID'] != '322656']

df_logs = df_logs.dropna()
df_logs = df_logs.drop_duplicates()
df_logs = df_logs[df_logs['Canvas CID'] != '322656']


df_grades = df_grades.dropna()
df_grades = df_grades.drop_duplicates()
df_grades = df_grades[df_grades['Canvas CID'] != '322656']


df_grades = df_grades[['Canvas CID', 'Uniqname', 'Final Grade', 'Term Code']]

df_grades['Course GPA'] = ((df_grades['Final Grade'].astype(int)/10-5).astype(int) +
                              (((df_grades['Final Grade'].astype(int)%10+.5)*.3).astype(int)-1)*.33)

df_grades['Course GPA'] = df_grades['Course GPA'].clip(0,4)
# print(df_main.columns)
# print(df_main.head(6))

# print(df_grades)
# print(df_grades.columns)
# print(df_grades[['Course GPA']].describe())

df_main_w19 = df_main[df_main['Term Code'] == '201920']

# print(df_main_w19['Canvas CName'].unique())

# this command joins the main df with the course map df, so course ID may be consistent
# df_combo = df_main.join(df_cmap.set_index('Temp CID'), on='Temp CID', how='left', lsuffix='', rsuffix=' map')

# df_smap.to_clipboard()
# df_cmap.to_clipboard()

# df_grades.to_clipboard()
# df_main.to_clipboard()
# df_logs.to_clipboard()


n_stud = df_main['Uniqname'].nunique()
print('df_main:',n_stud)
n_stud = df_grades['Uniqname'].nunique()
print('df_grades:',n_stud)
n_stud = df_logs['Uniqname'].nunique()
print('df_logs:',n_stud)
n_stud = df_smap['Uniqname'].nunique()
print('df_smap:',n_stud)
# print(df_main)
# print(df_logs)


n_stud_course_grades = df_grades.groupby(['Term Code','Canvas CID']).nunique()['Uniqname']
n_stud_course_main = df_main.groupby(['Term Code','Canvas CID']).nunique()['Uniqname']
n_stud_course_logs = df_logs.groupby(['Term Code']).nunique()['Uniqname']


distinct_uniqnames = df_main.Uniqname.unique()
distinct_uniqnames = pd.DataFrame(distinct_uniqnames)
# distinct_uniqnames = distinct_uniqnames.to_frame()
# distinct_uniqnames.to_clipboard()



n_stud_course_main.to_clipboard()
# n_stud_course_logs.to_clipboard()
# n_stud_course_grades.to_clipboard()






df_main: 3929
df_grades: 3865
df_logs: 1732
df_smap: 3961


In [75]:
df_main


,Uniqname,Sex,Intl/Dom,Class Level,Cum GPA,Canvas CName,Term Code,Canvas CID
1,abdsul,Male,International,Grad Mastr,3.673,HMP 654 001 FA 2019,202010,317173
2,aileenz,Female,Domestic,Grad Mastr,3.731,HMP 654 001 FA 2019,202010,317173
3,alexfox,Male,Domestic,Grad Mastr,3.877,HMP 654 001 FA 2019,202010,317173
4,angiemm,Female,Domestic,Grad Mastr,3.879,HMP 654 001 FA 2019,202010,317173
5,asbrandt,Male,Domestic,Grad Mastr,3.770,HMP 654 001 FA 2019,202010,317173
...,...,...,...,...,...,...,...,...
4583,vessecce,Male,Domestic,Sophomore,2.950,EECS 215 002 WN 2019,201920,268094
4584,xuanting,Male,Domestic,Junior,3.606,EECS 215 002 WN 2019,201920,268094
4585,yunzen,Male,International,Sophomore,3.430,EECS 215 002 WN 2019,201920,268094
4586,ziyangji,Male,International,Junior,3.837,EECS 215 002 WN 2019,201920,268094


In [28]:



# print(df_logs.head(6))
# reduce to 6 columns, sort, and remove canvas course id 322656
df_logs = df_logs[['Term Code','Uniqname','Click Timestamp', 'Canvas CID','Module','Click ID']]
df_logs = df_logs.sort_values(by=['Term Code','Uniqname','Click Timestamp'])
df_logs['Module'] = df_logs['Module'].replace({'VIEW_FILE_ACCESS': 'VFA',
                                                'VIEW_GRADE_DISTRIBUTION': 'VGD',
                                                'VIEW_ASSIGNMENT_PLANNING':'VAP',
                                                'VIEW_SET_DEFAULT':'VSD',
                                                'VIEW_RESOURCE_ACCESS':'VRA',
                                                'VIEW_ASSIGNMENT_PLANNING_WITH_GOAL_SETTING':'VAP-GS'
                                              })
# print(df_logs['Module'].unique())
df_logs['Click Date'] = pd.to_datetime(df_logs['Click Timestamp']).dt.date




# - - - - - - Dates for Term Start and End - - - - - - - 

# Winter 2019 - Start: 2019-01-09      End: 2019-05-02
# Fall 2019   - Start: 2019-09-03      End: 2019-12-20
# Winter 2020 - Start: 2020-01-08      End: 2020-04-30
# Fall 2020   - Start: 2020-08-31      End: 2020-12-18

# - - - - - - - - - - - - - - - - - - - - - - - - - - - -

df_logs['Term Start Date'] = df_logs['Term Code'].replace({
                                                '201920': '2019-01-09',
                                                '202010': '2019-09-03',
                                                '202020': '2020-01-08',
                                                '202110': '2020-08-31'
                                              })
df_logs['Term Start Date'] = df_logs['Term Start Date'].apply(pd.to_datetime).dt.date
df_logs['Term End Date'] = df_logs['Term Code'].replace({
                                                '201920': '2019-05-02',
                                                '202010': '2019-12-20',
                                                '202020': '2020-04-30',
                                                '202110': '2020-12-18'
                                              })
df_logs['Term End Date'] = df_logs['Term End Date'].apply(pd.to_datetime).dt.date
df_logs['Term Length'] = (df_logs['Term End Date']-df_logs['Term Start Date']).dt.days
# print(df_logs.columns)
# print(df_logs.head(6))




In [ ]:


df_click_times = df_logs[['Term Code','Term Start Date','Term End Date','Term Length','Click Date','Click ID']]
df_click_times = df_click_times.sort_values(by=['Term Code','Click Date'])
df_click_times['Days from Start'] = (df_click_times['Click Date']-df_click_times['Term Start Date']).dt.days
# df_click_times['Click Week'] = pd.to_datetime(df_click_times['Click Date']).dt.isocalendar().week
# print(df_click_times.columns)
# print(df_click_times.head(6))
df_click_times = df_click_times[['Term Code','Days from Start','Term Length']]
# print(df_click_times)
# print(df_click_times['Term Code'].unique())


# print(type(df_click_times['Click Date']))



# for each_term in df_click_times['Term Code'].unique() :
#     print(each_term)
#     print(get_term_desc(each_term))
#     df_click_times_subset = df_click_times[df_click_times['Term Code'] == each_term]
# #     min_date = df_click_times_subset['Days from Start']
# #     df_click_times_subset = df_click_times_subset['Term Code','Days from Start','Term Length']
#     print('min date:')
#     print(df_click_times_subset)
    
    
#     fig, axs = plt.subplots(figsize=(8, 1))
#     day_clicks = df_click_times_subset.groupby(['Click Week']).nunique()['Click ID'].plot(
#         kind='bar', rot=0, ax=axs
#     )
    
#     plt.xlabel(get_term_desc(each_term));

#     print(day_clicks)
#     print(each_term)


# # using subplot function and creating plot one
# plt.subplot(1, 2, 1)  # row 1, column 2, count 1
# plt.plot(x, y_1, 'r', linewidth=5, linestyle=':')
# plt.title('FIRST PLOT')
# plt.xlabel('x-axis')
# plt.ylabel('y-axis')
 
# # using subplot function and creating plot two
# # row 1, column 2, count 2
# plt.subplot(1, 2, 2)
 
# # g is gor green color
# plt.plot(x, y_2, 'g', linewidth=5)
# plt.title('SECOND PLOT')
# plt.xlabel('x-axis')
# plt.ylabel('y-axis')
 
# # space between the plots
# plt.tight_layout(4)
 
# # show plot
# plt.show()

# df_logs_combo = df_logs.join(df_cmap.set_index('Temp CID'),on='Temp CID', how='left', lsuffix='', rsuffix=' map')
# df_logs_combo = df_logs_combo[df_logs_combo['Canvas CID'] != '322656']

# df_logs_combo = df_logs_combo[['Term Code','Temp CID','Canvas CIDF','Campus ID']]
# print(df_log_combo.head(5))

# creates new column, showing Canvas ID when available, and showing Course ID when not (for Fall 2020 file)
# df_combo['Canvas CIDF'] = np.where(
#     df_combo['Canvas CID'].isna(), df_combo['Temp CID'], 
#      np.where(df_combo['Canvas CID'].notna(),  df_combo['Canvas CID'], 'Unknown')
# )

# reduce to 4 columns, sort, and remove canvas course id 322656
# df_combo = df_combo[['Term Code','Temp CID','Canvas CIDF','Campus ID']]
# df_combo = df_combo.sort_values(by=['Term Code','Canvas CIDF','Campus ID'])
# df_combo = df_combo[df_combo['Canvas CIDF'] != '322656']

# count students per course per term
# enrollments = df_combo.groupby(['Term Code','Canvas CIDF','Temp CID']).count()['Campus ID']

# myla_users = df_logs_combo.groupby(['Term Code','Canvas CID']).nunique()['Temp SID']
# myla_users = df_logs.groupby(['Term Code','Temp CID']).nunique()['Temp SID']

# print(enrollments.head(5))
# print(df_combo.columns)
# print(df_combo.head(5))


# print(df_logs.head(5))
# print(myla_users.head(5))



# to print dataframe as csv out to file on desktop, use following code
# out_file_path = Path('/Users/sticker/Desktop/myla_outputs/out.csv')
# out_file_path.parent.mkdir(parents=True, exist_ok=True)

# course_counts.to_csv(out_file_path)
# df_main.to_csv(out_file_path)
# df_combo.to_csv(out_file_path)
# df_cmap.to_csv(out_file_path)
# enrollments.to_csv(out_file_path)
# myla_users.to_csv(out_file_path)



#optional code bits

# this lists every unique value for the indicated column
# course_list = df_main.CanvasCourseID_long.unique()
# this counts every row for each value in the indicated column
# course_counts = df_main[['Term Code','Course ID']].value_counts()

# this lists every unique value for the indicated column
# course_term_list = df_main.CanvasCourseID_long.unique()
# this counts every row for each value in the indicated column
# course_term_counts = df_main[['Term Code']].value_counts()



# for col in df_main :
#     na_counts = df_main[col].isnull().sum()
#     print(col, " ",na_counts)

# print(df_main.nunique())
# print(course_list)
# print(course_counts)